# **Ejercicio 1: Preparar el entorno**

In [ ]:
!pip install mediapipe opencv-python-headless

# **Ejercicio 2: Cargar una imagen y detectar pose**

In [ ]:
# Cargar archivos desde tu computador a Colab
from google.colab import files
uploaded = files.upload()

# Obtén el nombre del primer archivo subido:
filename = next(iter(uploaded))

import cv2
import mediapipe as mp

mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True)

# Lee la imagen y verifica que exista
image = cv2.imread(filename)
if image is None:
    raise FileNotFoundError(f"No se pudo leer la imagen '{filename}'. Asegúrate de subirla y usar la ruta correcta.")

# Convierte de BGR (OpenCV) a RGB (MediaPipe)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Detecta pose
iresults = pose.process(image_rgb)

# Dibuja puntos y conexiones sobre la imagen original
mp.solutions.drawing_utils.draw_landmarks(
    image, iresults.pose_landmarks, mp_pose.POSE_CONNECTIONS)

# Muestra el resultado en Colab
from google.colab.patches import cv2_imshow
cv2_imshow(image)

# **Ejercicio 3: Video usando archivo (sin cámara)**

In [26]:
# Importar dependencias
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import cv2
import numpy as np
import mediapipe as mp

# Función: convertir respuesta JS a imagen OpenCV BGR
def js_to_image(js_reply):
    image_bytes = b64decode(js_reply.split(',')[1])
    jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
    img = cv2.imdecode(jpg_as_np, flags=1)
    return img

# JavaScript para inicializar y mostrar stream de cámara en Colab
def video_stream():
    js = Javascript('''
var video;
var div = null;
var stream;
var captureCanvas;
var pendingResolve = null;
var shutdown = false;

function removeDom() {
   stream.getVideoTracks()[0].stop();
   video.remove();
   div.remove();
   stream = null;
}

function onAnimationFrame() {
  if (!shutdown) window.requestAnimationFrame(onAnimationFrame);
  if (pendingResolve) {
    var result = "";
    if (!shutdown) {
      captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
      result = captureCanvas.toDataURL('image/jpeg', 0.8);
    }
    pendingResolve({img: result});
    pendingResolve = null;
  }
}

async function startStream() {
  if (div !== null) return;
  div = document.createElement('div');
  document.body.appendChild(div);
  video = document.createElement('video');
  video.setAttribute('playsinline', '');
  stream = await navigator.mediaDevices.getUserMedia({video: true});
  div.appendChild(video);
  video.srcObject = stream;
  await video.play();
  captureCanvas = document.createElement('canvas');
  captureCanvas.width = 640;
  captureCanvas.height = 480;
  window.requestAnimationFrame(onAnimationFrame);
}

startStream();
    ''')
    display(js)

# Función para capturar un frame vía JS
def video_frame():
    return eval_js('new Promise(resolve => { pendingResolve = resolve; })')

In [ ]:
# Inicia la transmisión de video
video_stream()

# Inicializar MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Procesamiento continuo
from google.colab.patches import cv2_imshow
while True:
    js_reply = video_frame()
    if not js_reply:
        break
    img = js_to_image(js_reply['img'])
    # Convertir BGR a RGB para MediaPipe
    rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb)
    # Dibujar landmarks
    mp.solutions.drawing_utils.draw_landmarks(
        img, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
    # Mostrar frame
    cv2_imshow(img)
    if cv2.waitKey(1) & 0xFF == 27:
        break

# Finalizar
pos.release()  # no aplica para JS, simplemente detener el loop
display(Javascript('window.stop()'))